In [1]:
import pandas as pd
import requests
import re

In [3]:
#df = pd.read_csv('ecoli_oxido_pdb.csv')[['gene','pdb','ligand']]
df = pd.read_csv('rossman_fba_results.merged - gene list.csv')

In [6]:
def get_pdb_codes(gene):
    url = 'http://rest.kegg.jp/get/' + gene
    r = requests.get(url)
    g = r.text.find('STRUCTURE') & r.text.find('PDB:')
    structs = []
    if g>0:
        structs = r.text[r.text.find('PDB:'):].split('\n')[0]
        structs = structs[5:].split(' ')
    return structs

In [7]:
def findligands(code):
    url = 'https://www.genome.jp/dbget-bin/www_bget?pdb:' + code
    r = requests.get(url)
    nadp_link = 'https://www.ebi.ac.uk/pdbe-srv/pdbechem/chemicalCompound/show/NAP'
    nadph_link = 'https://www.ebi.ac.uk/pdbe-srv/pdbechem/chemicalCompound/show/NPD'
    nad_link = 'https://www.ebi.ac.uk/pdbe-srv/pdbechem/chemicalCompound/show/NAD'
    nadh_link = 'https://www.ebi.ac.uk/pdbe-srv/pdbechem/chemicalCompound/show/NAI'
    
    nadp_res = r.text.find(nadp_link)
    nadph_res = r.text.find(nadph_link)
    nad_res = r.text.find(nad_link)
    nadh_res = r.text.find(nadh_link)
    
    r2 = list(zip([nadp_res,nadph_res,nad_res,nadh_res],['NADP','NADPH','NAD','NADH']));
    res_l = []
    for r in r2:
        if r[0] > 0:
            res_l.append(r[1])
    if len(res_l) < 1:
        return 'None';
    else:
        return ";".join(res_l)
    

In [8]:
def ko2genes(ko):
    url = 'http://rest.kegg.jp/link/genes/' + ko
    r = requests.get(url)
    geneKO = pd.DataFrame([x.split('\t') for x in r.text.split('\n')],columns=['ko','gene'])
    geneKO = geneKO.iloc[0:-1]
    return geneKO

In [13]:
def gene2ko(gene):
    url = 'http://rest.kegg.jp/link/ko/' + gene
    r = requests.get(url)
    geneKO = pd.DataFrame([x.split('\t') for x in r.text.split('\n')],columns=['gene','ko'])
    geneKO = geneKO.iloc[0:-1]
    return geneKO

In [80]:
all_results = []
gene_ko_data = []
for ko in df['KO'].unique():
    url = 'http://rest.kegg.jp/link/genes/' + ko
    r = requests.get(url)
    geneKO = pd.DataFrame([x.split('\t') for x in r.text.split('\n')],columns=['ko','gene'])
    geneKO = geneKO.iloc[0:-1]
    gene_ko_data.append(geneKO)
    
    for j,row2 in geneKO.iterrows():
        codes = get_pdb_codes(row2.gene)
        if len(codes)>0:
            r2 = pd.DataFrame({'gene':row2.gene, 'pdb': codes})
            r2['ko'] = ko
            all_results.append(r2)
    
    print('done with ko: ' + ko)

KeyboardInterrupt: 

In [104]:
all_results = []
gene_ko_data = []
for idx,row in df.iterrows():
    gene = 'eco:' + row['gene id']
    codes = get_pdb_codes(gene)
    if len(codes)>0:
        r2 = pd.DataFrame({'gene':gene, 'pdb': codes})
        ligands = []
        for code in codes:
            ligand= findligands(code)
            ligands.append(ligand)
        
        r2['ligand'] = ligands
        all_results.append(r2)

    print('done with gene: ' + gene)
    
results = pd.concat(all_results,axis=0)

done with gene: eco:b0002
done with gene: eco:b0031
done with gene: eco:b0031
done with gene: eco:b0116
done with gene: eco:b0173
done with gene: eco:b0243
done with gene: eco:b0311
done with gene: eco:b0347
done with gene: eco:b0351
done with gene: eco:b0356
done with gene: eco:b0386
done with gene: eco:b0425
done with gene: eco:b0509
done with gene: eco:b0529
done with gene: eco:b0596
done with gene: eco:b0662
done with gene: eco:b0888
done with gene: eco:b1033
done with gene: eco:b1093
done with gene: eco:b1109
done with gene: eco:b1210
done with gene: eco:b1288
done with gene: eco:b1300
done with gene: eco:b1380
done with gene: eco:b1385
done with gene: eco:b1395
done with gene: eco:b1478
done with gene: eco:b1479
done with gene: eco:b1525
done with gene: eco:b1539
done with gene: eco:b1606
done with gene: eco:b1692
done with gene: eco:b1746
done with gene: eco:b1761
done with gene: eco:b1779
done with gene: eco:b1852
done with gene: eco:b2020
done with gene: eco:b2028
done with ge

In [11]:
df = pd.read_csv('ecoli_oxido_pdb.csv')[['gene','pdb','ligand']]

In [54]:
gene = 'eco:b4267'
df[df.gene.isin([gene])]

,gene,pdb,ligand
142,eco:b4267,6DKH,None


In [222]:
#gene2ko(gene)

gene = 'eco:b3972'
codes = get_pdb_codes(gene)
ligands = []
for code in codes:
    ligand= findligands(code)
    ligands.append(ligand)

In [223]:
pd.DataFrame({'code': codes, 'ligand':ligands})

,code,ligand
0,1UXY,None
1,2MBR,None
2,1MBB,None
3,2Q85,None
4,1MBT,None


In [224]:
",".join(codes)

'1UXY,2MBR,1MBB,2Q85,1MBT'

In [133]:
",".join(df[df.gene.isin([gene])].pdb.tolist())

'1T4B,1T4D,1BRM,1GL3'

In [232]:
ko = 'K23007'
gl = ko2genes(ko)

In [233]:
len(gl)

182

In [234]:
gdf = []
dfs = []
codeDict = []

for gene in gl.gene.tolist():
    codes = get_pdb_codes(gene)
    coenzyme = []
    c = []
    for code in codes:
        if code not in codeDict:
            codeDict.append(code)
            c.append(code)
            coenzyme.append(findligands(code))
        
    dff = pd.DataFrame({'gene': gene, 'code': c,'coenzyme':coenzyme})
    dfs.append(dff)

In [235]:
z = pd.concat(dfs,axis=0)

In [236]:
z

,gene,code,coenzyme
